In [2]:
import os
import duckdb
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text


### Import Utilities in Notebooks
---
Since notebooks might be in different directories or subfolders, we can use pathlib to dynamically add our code folder to the Python search path. This ensures we can always find our utils.py.

In [3]:
import sys
from pathlib import Path

# Identify the project root and add the 'code' folder to sys.path
# This works even if your notebook is moved to a subfolder
project_root = Path.cwd().parent  # Adjust .parent if your notebook is deeper
code_path = project_root / "code"

if str(code_path) not in sys.path:
    sys.path.append(str(code_path))

# Now you can import your shared decorator
from utils import time_operation


### 1. Make connection to the database

---

The following environment variables are stored in .env file.</br>
DB_HOST=localhost</br>
DB_NAME=mcda5580</br>
DB_PORT=3306</br>
DB_USER=`<username>`</br>
DB_PWD=`<password>`</br>


In [4]:
# 1. Load the same .env variables we used for sqlalchemy
load_dotenv()

# 2. Build the DuckDB-specific MySQL connection string
# DuckDB uses a key-value pair format for MySQL
mysql_config = (
    f"host={os.getenv('DB_HOST')} "
    f"user={os.getenv('DB_USER')} "
    f"password={os.getenv('DB_PWD')} "
    f"database={os.getenv('DB_NAME')} "
    f"port={os.getenv('DB_PORT')}"
)


In [5]:
# 3. Initialize DuckDB, install/load extension, and ATTACH
con = duckdb.connect()
con.execute("INSTALL mysql; LOAD mysql;")
con.execute(f"ATTACH '{mysql_config}' AS mysql_db (TYPE MYSQL);")

print("DuckDB successfully attached to your MySQL database!")


DuckDB successfully attached to your MySQL database!


### 2. Extracting the Baskets
---

We will use DuckDB's list() and list_distinct() functions to roll up the milestones. This replaces the complex GROUP_CONCAT logic usually required in MySQL.

In [6]:
from utils import time_operation

# Define a wrapped function for DuckDB to Pandas conversion
@time_operation
def fetch_baskets(query, connection):
    return connection.execute(query).df()

# 1. Execute and Time: User-Level Baskets
user_query = """
    SELECT user_id, list_distinct(list(milestone_name)) as basket
    FROM mysql_db.rawdataDec15
    GROUP BY user_id
    HAVING len(basket) > 1
"""
df_user_basket, user_time = fetch_baskets(user_query, con)

# 2. Execute and Time: Session-Level Baskets
session_query = """
    SELECT user_id, date, list_distinct(list(milestone_name)) as basket
    FROM mysql_db.rawdataDec15
    GROUP BY user_id, date
    HAVING len(basket) > 1
"""
df_session_basket, session_time = fetch_baskets(session_query, con)

# Display results
print(f"User Baskets: {len(df_user_basket)} loaded in {user_time:.2f} ms")
print(f"Session Baskets: {len(df_session_basket)} loaded in {session_time:.2f} ms")


User Baskets: 2352 loaded in 1170.89 ms
Session Baskets: 20772 loaded in 1002.82 ms


### 3. Save the DataFrames
---
Save the DataFrames to results directory for futher proceedings. 

In [7]:
# 1. Identify the project root (one level up from the 'code' folder)
# Path.cwd() is the 'code' directory if the notebook is running there
project_root = Path.cwd().parent 

# 2. Define the target results directory at the root level
results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

# 3. Define file paths
user_pickle = results_dir / "user_baskets.pkl"
session_pickle = results_dir / "session_baskets.pkl"

# 4. Save the DataFrames
df_user_basket.to_pickle(user_pickle)
df_session_basket.to_pickle(session_pickle)

print(f"Baskets successfully stored in project results: {results_dir}")


Baskets successfully stored in project results: /home/bhavik/Dropbox/edu/smu/winter/data_mining/a3_association/5580-dm-a3-association/results


In [ ]:
df_user_basket.sample(5)


,user_id,basket
90,8028215,"[ReportsTab, ManageTab]"
2020,8030312,"[SendNow, ReSchedProj, ProjPreview, ReEditProj..."
2317,8030553,"[SendNow, SELinkElements, SetCompanyDefaul, Si..."
1936,8025688,"[ReportsTab, SendNow, SubjectMergeTags, ProjPr..."
1263,6242845,"[SaveCrossMarkete, CreateCrossMarke, ManageTab..."


In [ ]:
df_session_basket.sample(5)


,user_id,date,basket
13852,8004881,2015-10-02,"[SendNow, ProjPreview, AdvProjCreate, ManageTab]"
7614,8004877,2015-12-08,"[ManageTab, SendNow, ProjPreview, AdvProjCreate]"
5472,5130931,2015-09-25,"[OpenReportList, OpenReport, UnopenedData, Rep..."
14062,5741838,2015-11-24,"[SEDragResize, TxtIndentJustify, TxtStylesForm..."
15055,5775938,2015-08-27,"[SendNow, ReSchedProj, CaptionedImages, Insert..."
